## Loading modules and packages

In [2]:
import os

In [27]:
os.chdir("../bookstore/db/etl/data_preperation")

In [16]:
# from data_generator import generate_customer, generate_orders
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

## Cleaning the scraped books.csv

In [2]:
# Load the "books.csv" file into a Pandas DataFrame
books_df = pd.read_csv("books.csv")

# Generate unique book IDs for each row
books_df['book_id'] = range(1, len(books_df) + 1)

# Drop the "Age License" column
books_df = books_df.drop("Age License", axis=1)

# Define a function to extract the numeric value from the 'price' column
def extract_numeric_price(price_str):
    try:
        # Split the string by space and take the first part (the numeric value)
        numeric_part = price_str.split()[0]
        # Convert the numeric part to a float
        numeric_value = float(numeric_part)
        return numeric_value
    except:
        # If conversion fails, return None
        return None

# Apply the function to convert the 'price' column
books_df['Price'] = books_df['Price'].apply(extract_numeric_price)

In [15]:
books_df = pd.read_csv("C:/Users/Gor Mkrtchyan/OneDrive/Desktop/Marketing_Analytics_Group3/data/books.csv")

In [17]:
genres = ['Fiction', 'Non-Fiction', 'Mystery', 'Science Fiction', 'Fantasy', 'Romance', 'Thriller', 'Biography', 'History', 'Self-Help']

books_df['Genre'] = np.random.choice(genres, size=len(books_df))
books_df['Rating'] = np.round(np.random.uniform(0, 5, size=len(books_df)), 1)

In [23]:
# Save the updated DataFrame back to the "books.csv" file
books_df.to_csv("C:/Users/Gor Mkrtchyan/OneDrive/Desktop/Marketing_Analytics_Group3/data/books.csv", index=False)

## Extracting Authors Data from Scraped books.csv

In [3]:
# Create a DataFrame for the "Authors" table with unique author names
authors_df = pd.DataFrame({'full_name': books_df['Author'].unique()})

# Add an 'Author_ID' column with unique values
authors_df['author_id'] = range(1, len(authors_df) + 1)

# Save the "Authors" DataFrame to a new CSV file
authors_df.to_csv("authors.csv", index=False)

## Extracting Publishers Data from Scraped books.csv

In [4]:
# Create a DataFrame for the "Publishers" table with unique author names
publishers_df = pd.DataFrame({'name': books_df['Publisher'].unique()})

# Add an 'Publisher_ID' column with unique values
publishers_df['publisher_id'] = range(1, len(publishers_df) + 1)

# Save the "Publishers" DataFrame to a new CSV file
publishers_df.to_csv("publishers.csv", index=False)

## Changing the Names of Authors and Publishers to IDs in books.csv

In [6]:
books_df = pd.read_csv("books.csv")
authors_df = pd.read_csv("authors.csv")
publishers_df = pd.read_csv("publishers.csv")

# Perform the mapping of Author and Publisher columns to their respective IDs

books_df = books_df.merge(authors_df, left_on="Author", right_on="full_name", how="left")
books_df = books_df.merge(publishers_df, left_on="Publisher", right_on="name", how="left")

# Dropping the necessary columns

books_df.drop(columns=["Author","Publisher", "full_name", "name"], inplace=True)

# Renaming the columns

books_df.rename(columns={
    "Title": "title",
    "Price": "price",
    "ISBN": "isbn",
    "Publishing Year": "publication_year",
    "Language": "language",
    "Cover Type": "cover_type",
    "Pages Number": "pages_number"
}, inplace=True)

books_df.to_csv("books.csv", index=False)

## Creating and Generating data for the Inventory Table

In [7]:
books_df = pd.read_csv("books.csv")

# Create an "Inventory" DataFrame with the "book_id" column from "books.csv"
inventory_df = pd.DataFrame({'book_id': books_df['book_id']})

# Generate random values for the "stocklevel_used" and "stocklevel_new" columns
inventory_df['stocklevel_used'] = [random.randint(0, 100) for _ in range(len(inventory_df))]
inventory_df['stocklevel_new'] = [random.randint(0, 100) for _ in range(len(inventory_df))]

# Save the "Inventory" DataFrame to a new CSV file
inventory_df.to_csv("Inventory.csv", index=False)

## Generating Customer Data

In [8]:
NUMBER_OF_CUSTOMERS=2500

customer_data = [generate_customer(customer_id) for customer_id in range(NUMBER_OF_CUSTOMERS)] 
pd.DataFrame(customer_data).to_csv('customers.csv', index=False)

## Creating and Generating data for the OrderItem Table

In [9]:
books_df = pd.read_csv("books.csv")

# Generate 4000 unique order IDs
unique_order_ids = [str(i) for i in range(1, 4001)]

# Generate 2000 random order IDs ensuring at least one occurrence of each unique order ID
remaining_order_ids = random.choices(unique_order_ids, k=2000)
order_ids = unique_order_ids + remaining_order_ids

# Shuffle the order IDs to randomize the order
random.shuffle(order_ids)

# Create the "OrderItem" DataFrame
order_item_df = pd.DataFrame({'order_id': order_ids})

# Select random book IDs from the "books.csv" file
order_item_df['book_id'] = random.choices(books_df['book_id'], k=6000)

# Generate random quantities (1-3)
order_item_df['quantity'] = [random.randint(1, 3) for _ in range(6000)]

# Assign the book's price to the "price" column based on the selected book_id
order_item_df['price'] = order_item_df['book_id'].apply(
    lambda book_id: books_df.loc[books_df['book_id'] == book_id, 'price'].values[0]
)

# Save the "OrderItem" DataFrame to a new CSV file
order_item_df.to_csv("OrderItem.csv", index=False)

## Creating and Generating data for the Orders Table

In [11]:
order_item_df = pd.read_csv("OrderItem.csv")
customers_df = pd.read_csv("customers.csv")

# Function to generate a random date between two dates
def random_date(start_date, end_date):
    days_diff = (end_date - start_date).days
    random_days = random.randint(0, days_diff)
    return start_date + timedelta(days=random_days)

# Define the date range for order_date
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 10, 28)

# Create the "Orders" DataFrame
orders_df = pd.DataFrame()

# Generate unique order IDs based on the "OrderItem.csv" file
orders_df['order_id'] = order_item_df['order_id'].unique()

# Randomly select customer IDs from the "customers.csv" file
orders_df['customer_id'] = random.choices(customers_df['customer_id'], k=len(orders_df))

# Generate random order dates between 01.01.2022 and 28.10.2023
orders_df['order_date'] = [random_date(start_date, end_date) for _ in range(len(orders_df))]

# Calculate the subtotal by grouping "OrderItem.csv" by "order_id" and summing "quantity * price"
subtotal_df = order_item_df.groupby('order_id')['quantity'].sum().reset_index()
subtotal_df = subtotal_df.rename(columns={'quantity': 'total_quantity'})
subtotal_df = subtotal_df.merge(order_item_df.groupby('order_id')['price'].mean().reset_index(), on='order_id')

orders_df = orders_df.merge(subtotal_df, on='order_id', how='left')
orders_df['subtotal'] = orders_df['total_quantity'] * orders_df['price']

# Generate random shipping costs between 500 and 2000
orders_df['shipping'] = [random.randint(500, 2000) for _ in range(len(orders_df))]

# Calculate the total by summing the subtotal and shipping
orders_df['total'] = orders_df['subtotal'] + orders_df['shipping']

# Dropping the necessary columns
orders_df.drop(columns=["total_quantity","price"], inplace=True)

# Save the "Orders" DataFrame to a new CSV file
orders_df.to_csv("Orders.csv", index=False)